In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
colab = False

if colab:
    from google.colab import drive
    drive.mount('/content/drive')
    os.chdir('/content/drive/MyDrive/Tesis/tesis/4_classifier')
else:
    os.chdir('../4_generative')

In [3]:
import torch
import pandas as pd

## Functions

In [ ]:
import librosa
import soundfile as sf

def load_audio(name, verbose=False):
    '''
    Load audio file and return it as a numpy array

    Parameters:
    name (str): Name of the audio file

    Returns:
    audio (np.array): Audio file as a numpy array
    '''
    audio, sr = librosa.load(f'../data/train_gen/BOAFAB/{name}', sr=16_000)

    return audio, sr

def save_audios(dict_audios, real_fake, sr):
    '''
    Save audios in the path ../data/embedding/BOAFAB_{real_fake}/

    Parameters:
    dict_audios (dict): Dictionary with the audios to save
    real_fake (str): 'real' or 'fake'
    sr (int): Sample rate
    '''
    os.makedirs(f"../../data/embedding/BOAFAB_{real_fake}/", exist_ok=True)

    for key in dict_audios:
        audio = dict_audios[key]
        
        new_path = f'{key}.wav'
        sf.write(f'../data/embedding/BOAFAB_{real_fake}/{new_path}', audio, sr)

# Create samples


In [ ]:
import json

device = "cuda" if torch.cuda.is_available() else "cpu"
model_diff = torch.load(f'./models_trained/DIFFUSION/75_DIFFUSION_model.pth', map_location=device)
model_diff.eval();

from sampling.sampling_utils import get_diffusion_sample
import sys
sys.path.append(os.path.abspath("../model_utils"))
from model_utils.diffusion_utils import calc_diffusion_hyperparams
with open('./config.json', 'r') as f:
    config = json.load(f)
    sr = config['audio_config']['sample_rate']

diffusion_config = config['diffusion_config']

to_save = {}
for i in range(5):
  diffusion_hyperparams = calc_diffusion_hyperparams(**diffusion_config)
  samples_diff = get_diffusion_sample(model_diff, (50, 1, sr), diffusion_hyperparams)

  sample_diff = samples_diff.cpu()

  to_save[f'{i}_samples_embedding'] = samples_diff

import pickle
with open('../4_systematic_selection/4000_samples.pickle', 'wb') as f:
    pickle.dump(to_save, f)

## Processing 

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
clf_model = torch.load('../3_classifier/1711206440837-resnext29_8_64_sgd_plateau_bs96_lr1.0e-02_wd1.0e-03-best-acc.pth', map_location=device)

In [5]:
import pickle 

# samples = torch.load('../figuras/2000_samples.pickle', map_location=torch.device('cpu'))
with open('../4_systematic_selection/4000_samples.pickle', 'rb') as f:
    samples = pickle.load (f)

samples = torch.cat(tuple(samples.values()), axis=0)

In [6]:
dict_to_save = {
    i + 1690: audio[0].numpy() for i, audio in enumerate(samples)
}
save_audios(dict_to_save, 'GENERATED', 16_000)

In [21]:
list_real_paths = os.listdir('../../data/train_gen/BOAFAB')
list_real_paths = pd.Series(list_real_paths).sample(n=1200, random_state=123)

In [46]:
dict_to_save = {}
for i, path in enumerate(list_real_paths):
    audio, sr = load_audio(path)
    dict_to_save[i] = audio

save_audios(dict_to_save, 'REAL', 16_000)

In [47]:
save_audios(dict_to_save, 'REAL', 16_000)

In [44]:
from IPython.display import Audio
Audio(data=audio, rate=sr)